In [ ]:
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
import random
import os

# File paths
TRAIN_DATADIR = '/home/kaushal/Datasets/DistractedDriverDetectionData/train/'
TEST_DATADIR = '/home/kaushal/Datasets/DistractedDriverDetectionData/test/'
TRAIN_DATA_NPY = './npy_arrays/train_data.npy'
TRAIN_DATA_COLOR_NPY = './npy_arrays/train_color_data.npy'
TEST_DATA_NPY = './npy_arrays/test_data.npy'
TEST_UNKNOWN = './datasets/testunknown'

# Categorical information
CATEGORIES = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
IMG_CATEGORIES = ['Safe Driving',
  'Texting Right' ,
  'Talking Right' , 
  'Texting Left ' ,
  'Talking Left ' ,
  'Adjust Radio/Music Player' , 
  'Drinking',
  'Reaching Behind', 
  'Hair and Makeup' , 
  'Talking to Passenger']

# Model information
IMG_SIZE = 128
LR = 1e-3
MODEL_NAME = 'Distracted_Driver_CNN' 

# Prediction setup
model = load_model('./assets/DDD.model/')
cv2Font = cv2.FONT_HERSHEY_SIMPLEX
prediction_queue = []
current_prediction = 'Safe Driving'
alert = False

def getArea(x, y, w, h):
    return (x+w) * (y+h)

def toString(n):
    return str(n)

def prepare(frame):
    img_array = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # convert to grayscale
#     plt.imshow(img_array, cmap="gray")
#     plt.show()
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize image to match model's expected sizing
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1)/255.0  # return the image with shaping that TF wants.

def add_to_prediction_queue(prediction, current_prediction, alert):
    category = np.argmax(prediction)
    if len(prediction_queue) >= 10:
        prediction_queue.pop(0)
    
    print("Prediction:", prediction[category], IMG_CATEGORIES[category])
    
    if prediction[category] > 0.60:
        prediction_queue.append(category)
        current_prediction = toString(IMG_CATEGORIES[category])
    else:
        prediction_queue.append(0)
        current_prediction = toString(IMG_CATEGORIES[0])
        
    print("Prediction Queue:", prediction_queue)
    
    if prediction_queue.count(0) < 4:
        alert = True
    else:
        alert = False

In [21]:
# capture frames from a video
vc = cv2.VideoCapture('./assets/test.mp4')
if vc.isOpened():
    rval, frame = vc.read()
else:
    rval = False

# loop runs if new frame exists
while rval:
    # read frame from a video
    rval, frame = vc.read()
    if not rval:
        continue
    
    image = prepare(frame)
    prediction = model.predict([image])
    
    add_to_prediction_queue(prediction[0], current_prediction, alert)
    
#     print("Prediction:", IMG_CATEGORIES[np.argmax(prediction)])
    
    cv2.putText(frame, current_prediction, (50,50), cv2Font, 0.9, (0, 0, 255), 1)
    print("Current Prediction:", current_prediction)

    if alert == True:
        cv2.putText(frame, "ALERT", (450,50), cv2Font, 0.9, (0, 0, 255), 1)
    
    # Display frames in a window
    cv2.namedWindow('feed', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('feed', 1280, 750)
    cv2.imshow('feed', frame)
    
    # Wait for Esc key to stop
    if cv2.waitKey(33) == 27:
        break

# De-allocate any associated memory usage
cv2.destroyAllWindows()
vc.release()

Prediction: 0.59276646 Adjust Radio/Music Player
Prediction Queue: [0]
Current Prediction: Safe Driving
Prediction: 0.5937546 Adjust Radio/Music Player
Prediction Queue: [0, 0]
Current Prediction: Safe Driving
Prediction: 0.58941734 Adjust Radio/Music Player
Prediction Queue: [0, 0, 0]
Current Prediction: Safe Driving
Prediction: 0.85035014 Adjust Radio/Music Player
Prediction Queue: [0, 0, 0, 5]
Current Prediction: Safe Driving
Prediction: 0.8387914 Adjust Radio/Music Player
Prediction Queue: [0, 0, 0, 5, 5]
Current Prediction: Safe Driving
Prediction: 0.51333594 Texting Left 
Prediction Queue: [0, 0, 0, 5, 5, 0]
Current Prediction: Safe Driving
Prediction: 0.5118124 Texting Left 
Prediction Queue: [0, 0, 0, 5, 5, 0, 0]
Current Prediction: Safe Driving
Prediction: 0.7410362 Adjust Radio/Music Player
Prediction Queue: [0, 0, 0, 5, 5, 0, 0, 5]
Current Prediction: Safe Driving
Prediction: 0.5368754 Adjust Radio/Music Player
Prediction Queue: [0, 0, 0, 5, 5, 0, 0, 5, 0]
Current Prediction

Prediction: 0.98333377 Texting Left 
Prediction Queue: [3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
Current Prediction: Safe Driving
Prediction: 0.96362424 Texting Left 
Prediction Queue: [3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
Current Prediction: Safe Driving
Prediction: 0.97988373 Texting Left 
Prediction Queue: [3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
Current Prediction: Safe Driving
Prediction: 0.87655735 Texting Left 
Prediction Queue: [3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
Current Prediction: Safe Driving
Prediction: 0.68234235 Texting Left 
Prediction Queue: [3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
Current Prediction: Safe Driving
Prediction: 0.8917768 Texting Left 
Prediction Queue: [3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
Current Prediction: Safe Driving
Prediction: 0.8868167 Texting Left 
Prediction Queue: [3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
Current Prediction: Safe Driving
Prediction: 0.9990044 Texting Left 
Prediction Queue: [3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
Current Prediction: Safe Driving
Prediction: 0.97336495 Texting Left 
Prediction Que